In [1]:
import pandas as pd
import sqlite3

In [2]:
engine = sqlite3.connect("Group_Project_3.db")

### Review the tables


In [3]:
query= "SELECT name FROM sqlite_master WHERE type='table'"
table= pd.DataFrame(engine.execute(query),columns=["project3_table"])
table

,project3_table
0,stock
1,Apple_price
2,Apple_volume
3,Google_price
4,Google_volume
5,Microsoft_price
6,Microsoft_volume
7,Intel_price
8,Intel_volume
9,Oracle_price


### Review the Apple_price table


In [27]:
table_name="Apple_price"
query=f"Select * from {table_name}"
value=engine.execute(query)
query=f"""SELECT name FROM PRAGMA_TABLE_INFO('{table_name}')"""
column=engine.execute(query)
clean_column=[]
for item in list(column):
    clean_column.append(item[0])
temp=pd.DataFrame(value,columns=clean_column)
temp["date"]=pd.to_datetime(temp["date"])
temp.set_index("date",inplace=True)
temp

,stock_id,Open,High,Low,Close,Month,normalization,Percentage_change,accumulation
date,,,,,,,,,
2023-01-03,AAPL,130.28,130.90,124.17,125.07,1,1.0000,NaN,NaN
2023-01-04,AAPL,126.89,128.66,125.08,126.36,1,1.0103,0.0103,1.0103
2023-01-05,AAPL,127.13,127.77,124.76,125.02,1,0.9996,-0.0106,0.9996
2023-01-06,AAPL,126.01,130.29,124.89,129.62,1,1.0364,0.0368,1.0364
2023-01-09,AAPL,130.47,133.41,129.89,130.15,1,1.0406,0.0041,1.0406
...,...,...,...,...,...,...,...,...,...
2023-12-22,AAPL,195.18,195.41,192.97,193.60,12,1.5479,-0.0055,1.5479
2023-12-26,AAPL,193.61,193.89,192.83,193.05,12,1.5435,-0.0028,1.5435
2023-12-27,AAPL,192.49,193.50,191.09,193.15,12,1.5443,0.0005,1.5443


### Create a function to work for each table

In [28]:
def Database_to_Frame(table_name):
    query=f"Select * from {table_name}"
    value=engine.execute(query)
    query=f"""SELECT name FROM PRAGMA_TABLE_INFO('{table_name}')"""
    column=engine.execute(query)
    clean_column=[]
    for item in list(column):
        clean_column.append(item[0])
    temp=pd.DataFrame(value,columns=clean_column)
    temp["date"]=pd.to_datetime(temp["date"])
    temp.set_index("date",inplace=True)
    return temp

In [29]:
Google=Database_to_Frame("Google_price")
Google

,stock_id,Open,High,Low,Close,Month,normalization,Percentage_change,accumulation
date,,,,,,,,,
2023-01-03,GOOGL,89.59,91.05,88.52,89.12,1,1.0000,NaN,NaN
2023-01-04,GOOGL,90.35,90.65,87.27,88.08,1,0.9883,-0.0117,0.9883
2023-01-05,GOOGL,87.47,87.57,85.90,86.20,1,0.9672,-0.0213,0.9672
2023-01-06,GOOGL,86.79,87.69,84.86,87.34,1,0.9800,0.0132,0.9800
2023-01-09,GOOGL,88.36,90.05,87.86,88.02,1,0.9877,0.0078,0.9877
...,...,...,...,...,...,...,...,...,...
2023-12-22,GOOGL,140.77,141.99,140.71,141.49,12,1.5876,0.0076,1.5876
2023-12-26,GOOGL,141.59,142.68,141.19,141.52,12,1.5880,0.0002,1.5880
2023-12-27,GOOGL,141.59,142.08,139.89,140.37,12,1.5751,-0.0081,1.5751


In [33]:
Google_volume=Database_to_Frame("Google_volume")
Google_volume

,stock_id,Adj Close,Volume
date,,,
2023-01-03,GOOGL,89.120003,28131200
2023-01-04,GOOGL,88.080002,34854800
2023-01-05,GOOGL,86.199997,27194400
2023-01-06,GOOGL,87.339996,41381500
2023-01-09,GOOGL,88.019997,29003900
...,...,...,...
2023-12-22,GOOGL,141.490005,26514600
2023-12-26,GOOGL,141.520004,16780300
2023-12-27,GOOGL,140.369995,19628600
